In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

22/08/17 06:14:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### To create 2 udf

1. create your function
2. convert that function to udf using udf function
3. access the udf function in your spark session

In [4]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

In [3]:
def my_len(s):
    return len(s)

In [5]:
len_udf = udf(my_len, IntegerType())

In [6]:
df = spark.createDataFrame(
    [
        (1, "Peter Pan", "1999-04-23", 90, 3),
        (2, "Alice Smith", "2000-05-15", 100, 2),
        (3, "Bob Miller", "2000-09-01", 80, 3),
        (4, "April May", "1999-09-30", 110, 1),
        (5, "Billie Jean", "1990-07-12", 360, None)
    ],
    ["leftid", "name", "date-of-birth", "creditscore", "universityid"]
)

In [8]:
df.select(len_udf("name")).show()

+------------+
|my_len(name)|
+------------+
|           9|
|          11|
|          10|
|           9|
|          11|
+------------+



In [9]:
@udf(returnType=IntegerType())
def decorator_len(s):
    return len(s)

In [10]:
df.select(decorator_len("name")).show()

+-------------------+
|decorator_len(name)|
+-------------------+
|                  9|
|                 11|
|                 10|
|                  9|
|                 11|
+-------------------+



In [16]:
from pyspark.sql.functions import PandasUDFType, pandas_udf
import pandas as pd

In [17]:
@pandas_udf(IntegerType(), PandasUDFType.SCALAR)
def slen(s: pd.Series):
    return s.str.len()

/Users/avinashs/Downloads/softwares/spark-3.0.0-bin-hadoop2.7/python/pyspark/sql/pandas/functions.py:383: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [ ]:
df.select(slen("name")).show()

In [11]:
spark.stop()